In [ ]:
import pandas as pd

# Cargar el DataFrame (asegúrate de incluir las columnas necesarias)

df_final = pd.read_csv('df_final11.csv')

df_final['date'] = pd.to_datetime(df_final['ts'])

    # Extraer el año
df_final['year'] = df_final['date'].dt.year

    # Eliminar zona horaria de 'date'
df_final['date'] = df_final['date'].dt.tz_localize(None)

df_final['artist'] = df_final['master_metadata_album_artist_name']

df_final['song'] = df_final['master_metadata_track_name']

df_final['min_played'] = df_final['ms_played'] / 60000

df_final['month'] = df_final['date'].dt.month


    # Definir rango de fechas
fecha_inicio = pd.Timestamp("2017-01-01")
fecha_fin = pd.Timestamp("2024-12-29")

    # Filtrar el DataFrame dentro del rango de fechas
df_final = df_final[(df_final['date'] >= fecha_inicio) & (df_final['date'] <= fecha_fin)]

    # Agrupar los datos por año y mes
df_grouped = df_final.groupby(['year', 'month'])['min_played'].sum().reset_index()

artist_years = df_final.groupby('artist')['year'].nunique().reset_index()
artist_years = artist_years.rename(columns={'year': 'years_active'})

    # Filtrar artistas con años activos = 8
consistent_artists = artist_years[artist_years['years_active'] == 8]

    # Calcular el total de minutos reproducidos por artista
artist_playtime = df_final.groupby('artist')['min_played'].sum().reset_index()
artist_playtime = artist_playtime.rename(columns={'min_played': 'total_min_played'})

    # Unir con los artistas consistentes
consistent_artists = consistent_artists.merge(artist_playtime, on='artist')

    # Ordenar por minutos reproducidos y tomar el top N



# seleccionar solo las variables a utilizar

df_final = df_final[['date', 'year', 'artist', 'song', 'ms_played', 'min_played', 'ts', 'month']]

# exportar df_final

df_final.to_csv('df_final1.csv', index=False)

In [7]:

import pandas as pd
df_final = pd.read_csv('df_final.csv')

In [ ]:
print(df_final.head())
print(df_final.dtypes)


In [ ]:
from pandasgui import show

show(df_final)

In [10]:
#  porcentaje de subida de minutos escuchados de Mora de 2024 respecto al 2023

# Filtrar los datos de Mora

mora_data = df_final[df_final['artist'] == 'Mora']

# Agrupar por año y sumar los minutos reproducidos

mora_grouped = mora_data.groupby('year')['min_played'].sum().reset_index()

# Filtrar los años 2023 y 2024

mora_2023 = mora_grouped[mora_grouped['year'] == 2023]['min_played'].values[0]


mora_grouped


,year,min_played
0,2020,3.409483
1,2021,0.219533
2,2022,28.660250
3,2023,2287.860483
4,2024,8957.109733
